# Лекция 8: строки, кодировки и Python

## Разговор про строки в общем

Краткий конспект этой части:

* Символы и строки как абстрактное понятие без привязки к битам.
* Способы представления (Си-строки, длина и идея lenval)
* Хранение (сжатие, интернирование, cow, ropes, бор (trie), лексикографический порядок)
* Алгоритмы (поиск подстроки, расстояние между строками, поиск в множестве по совпадению точному и нет). 

Почитать:

* https://ru.wikipedia.org/wiki/Строковый_тип
* https://en.wikipedia.org/wiki/String_(computer_science)
* Билл Смит, "Методы и алгоритмы вычислений на строках"
* http://neerc.ifmo.ru/wiki/index.php?title=%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC%D1%8B_%D0%BD%D0%B0_%D1%81%D1%82%D1%80%D0%BE%D0%BA%D0%B0%D1%85
* http://russian.joelonsoftware.com/Articles/BacktoBasics.html
* https://queue.acm.org/detail.cfm?id=2010365

## Кодировки и стандарт Unicode

### Что такое кодировки и как появились

#### Что такое кодировка?

* В компьютере данные представлены в виде набора бит объединённых в байты.
* Биты и байты можно можно использовать для записи текста.
* Формат представления текста в бинарном виде называется кодировкой.

#### ASCII

* American Standard Code for Information
* 1968 год
* Определяла коды для символов, от 0 до 127.
* В основном состояла из строчных и прописных символов английского алфавита, цифр, пунктуации, математических символов и специальных управляющих кодов.
* 'a' = 97
* "a..z" (и "A..Z") идут подряд (не во всех кодировках так)

#### Локальные альтернативы ascii
* Американский стандарт, не было нужных символов с диакритикой (французский) или просто символов алфавита (русский). Какое-то время с этим мирились.
* Некоторые компании создавали свои альтернативы, но они не были общеприняты.
* В 1980 большинство персональных компьютеров имели 8-битные байты, поэтому можно было хранить значения 0..255.
* Альтернативные кодировки дополняли ascii дописывая используя вторую половину байта.
* Примеры: для русского (koi8-r, cp1251), французского (latin1) и т.д. 

#### Проблемы с однобайтовыми кодировками

* Сложно использовать несколько языков одновременно.
* Нетривиальность определения кодировок при совместном использовании (поэтому обычно использовали одну).
* Неправильное отображение при неправильном определении.
* Неуниверсальность записи различных данных.

### Стандарт Unicode

#####  Начало стандартизации

* В 1980-х люди хотели решить проблемы связанные с использованием однобайтных кодировок.
* Началось движение в сторону стандартизации общего Unicode формата представления и записи символов.
* Сначала хотели 16-битные символы, но оказалось мало.
* Решили формально не ограничивать пространство кодов (но по факту сейчас все символы помещаются в 4 байта и расширять пока не планируется).

##### Две основные части стандарта

* UCS (universal character set) - универсальный набор символов Unicode, задано соответствие между символом и кодом.
* UTF (unicode transformation format) - семейство кодировок определяет машинное представление последовательности кодов UCS.

##### Определения
* Символ (character) - наименьшая единица текста ('A', 'È', 'Ω' и ...).
* Точка кода (code point) - целое число из таблицы отображения символов на коды, представляет отдельный символ.
* Глиф (glyph) - графические элементы записи конкретного символа (обычно обрабатываются прозрачно для нас, заботу на себя берёт ОС, браузер, GUI).
* Юникод (Unicode) - стандарт кодирования символов, по сути представляет собой таблицу с описаниями символов, правила и рекомендации их представления и использования. 
* Кодировка - правила перевода символов Unicode в последовательность байт (UTF часть стандарта).

##### Детальнее про внутреннее представление

* https://ru.wikipedia.org/wiki/Юникод
* https://www.unicode.org/faq/

#### Кодировки и Unicode

* Кодировки могут быть постоянного и переменного размера.
* Постоянного:
  * latin1 (однобайтовая, не юникод)
  * UCS-2 (двухбайтовая; аналог UTF-16 без суррогатных пар)
  * UCS-4 (четырёхбайтовая, синоним UTF-32).
* Переменного:
  * UTF-8

#### UTF-16 и UTF-32

* 2 и 4 байтовые юникод кодировки.
* Фиксированной длинны.
* Важен порядок байт: BE (big-endian) или LE (little-endian).
* В начале файлов часто бывает символ BOM - byte order mark - обозначающий BE или LE.
* Из-за BOM могут возникать проблемы. Важно понимать, когда он есть и когда его убирать.
* А лучше использовать UTF-8.

#### UTF-8

##### Правила записи UTF-8

* Если код меньше 128, то просто записываем байт.
* Если код между 128 и 0x7ff, то записываем два байта со значениями между 128 и 255.
* Для кодов больше 0x7ff используются трёх и четырёх байтовые последовательности, каждый байт со значениями между 128 и 255.
* Теоретически, расширяется до 6 байт, но на практике (в текущем Unicode) хватает только 4 байт, т.к. нет символов с кодом больше 0x10FFFF).

Unicode UTF-8:
* 0x00000000 — 0x0000007F: 0xxxxxxx
* 0x00000080 — 0x000007FF: 110xxxxx 10xxxxxx
* 0x00000800 — 0x0000FFFF: 1110xxxx 10xxxxxx 10xxxxxx
* 0x00010000 — 0x001FFFFF: 11110xxx 10xxxxxx 10xxxxxx 10xxxxxx

Теоретически возможны, но не включены в стандарт также:

* 0x00200000 — 0x03FFFFFF: 111110xx 10xxxxxx 10xxxxxx 10xxxxxx 10xxxxxx
* 0x04000000 — 0x7FFFFFFF: 1111110x 10xxxxxx 10xxxxxx 10xxxxxx 10xxxxxx 10xxxxxx

Замечания:
* Несмотря на то, что UTF-8 позволяет указать один и тот же символ несколькими способами, только наиболее короткий из них правильный.
* Остальные формы должны отвергаться по соображениям безопасности.

##### Удобные свойства UTF-8
* Можно записать любой существующий символ Unicode.
* В записях utf-8 символов нет промежуточных нулевых байт, поэтому такие строки совместимы с C-функциями вроде strcpy и т.д., а также могут передаваться по протоколам не поддерживающим промежуточные нулевые байты.
* Строка ASCII - валидный UTF-8 текст.
* UTF-8 достаточно компактен: большинство используемых символов помещаются в 1-2 байтовые последовательности.
* В случае повреждения или потери данных можно определить начало следующего UTF-8 кода и продолжить с этого места (+ маловероятно, что случайные данные будут выглядеть как UTF-8).

## Работа с кодировками в Python

Работаем с кодировками когда:
* На входе и выходе данных программы.
* Форматы хранения.

### Байты и текст

* bytes - байты, строка как последовательности байт полученная в результате чтения из источников и/или кодирования/декодирования между форматами.
* str - строка, юникод-строка в кодировке utf-8.
* Оба типа immutable.

#### bytes
* b"..." - литерал для последовательности байт.
* bytes(iterable_of_ints) - последовательность чисел в последовательность байт.
* bytes(string, encoding[, errors]) - превратить str строку в байты.
* bytes(bytes_or_buffer) - неизменяемый вариант переданного буфера или копия bytes-объекта.
* bytes(int) - набор null-байтов длины входного int
* bytes() - пустой объект
* Есть редко нужный в повседневном использовании mutable-вариант bytearray.

In [1]:
b'abcdef'

b'abcdef'

In [2]:
b'\x65'

b'e'

In [3]:
type(b'\x65')

bytes

In [4]:
bytes("abcdef", encoding="utf-8")

b'abcdef'

#### str
* "..." или '...' или """...""" - строковый литерал.
* str(bytes_or_buffer[, encoding, errors]) - конвертировать байтовую последовательность в строку.
* str(object='') - превратить объект в строку (\_\_str\_\_, repr)

In [5]:
'abcdef'

'abcdef'

In [6]:
s = str('abcdef')

In [7]:
type(s)

str

In [8]:
str(str)

"<class 'str'>"

Аргумент errors определяет поведение при ошибке:
* 'strict' - (выбран по-умолчанию) выбрасывать исключение UnicodeDecodeError.
* 'replace' - замена проблемных символов на U+FFFD (‘REPLACEMENT CHARACTER’).
* 'ignore' - пропустить проблемный символ, не учитывать его в результирующей юникод записи.

In [9]:
str(b'\x80abc', errors='strict')  

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [10]:
str(b'\x80abc', errors='replace')

'�abc'

In [11]:
str(b'\x80abc', errors='ignore')

'abc'

Можно создавать односимвольные строки из чисел:

In [12]:
chr(40960)

'ꀀ'

И обратно:

In [13]:
ord('\ua000')

40960

### Кодирование и декодирование

* Кодировка в Python задаётся своим названием (таблица встроенных кодировок https://docs.python.org/3.5/library/codecs.html#standard-encodings).
* Методы encode (и decode) используют имя кодировки для явного указания целевой кодировки (кодировки источника).

#### .encode([encoding], [errors='strict'])

Конвертация str() строки в bytes() с указанной кодировкой encoding.

In [14]:
u = chr(40960) + 'abcd' + chr(1972)
print(u)

ꀀabcd޴


In [15]:
u.encode('utf-8')

b'\xea\x80\x80abcd\xde\xb4'

In [16]:
print([bin(elem) for elem in u.encode('utf-8')])

['0b11101010', '0b10000000', '0b10000000', '0b1100001', '0b1100010', '0b1100011', '0b1100100', '0b11011110', '0b10110100']


In [17]:
u.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\ua000' in position 0: ordinal not in range(128)

In [18]:
u.encode('ascii', 'ignore')

b'abcd'

In [19]:
u.encode('ascii', 'replace')

b'?abcd?'

In [20]:
u.encode('ascii', 'xmlcharrefreplace')

b'&#40960;abcd&#1972;'

#### .decode([encoding], [errors])

Конвертация bytes() в str() предполагая, что bytes() имеет кодировку encoding.

In [21]:
u = chr(40960) + u'abcd' + chr(1972) # Assemble a string

In [22]:
utf16_version = u.encode('utf-16') # Encode as UTF-16

In [23]:
type(utf16_version), utf16_version, len(utf16_version)

(bytes, b'\xff\xfe\x00\xa0a\x00b\x00c\x00d\x00\xb4\x07', 14)

Запись выше:
* 2 байта BOM
* 2 байта 40960 (\x00\xa0)
* 4 по 2 байта на символ (a\x00 и т.п.)
* 2 байта на 1972 (\xb4\x07)

In [24]:
"a".encode("utf-16")

b'\xff\xfea\x00'

In [25]:
u2 = utf16_version.decode('utf-16') # Decode using UTF-16

In [26]:
u == u2 # The two strings match

True

#### Низкоуровневые преобразования и работа с кодировками

* Можно найти в модуле codecs.
* Обычно, это не нужно и базовых достаточно.

### Строки и символы юникод-литералы

In [27]:
s = "a\xac\u1234\u20ac\U00008000"

* \x - два hex.
* \u - четыре hex.
* \U - восемь hex.

In [28]:
for character in s:
    print(ord(character), end=" ")

97 172 4660 8364 32768 

### codecs

* Иногда необходима низкоуровневая работа с байтами кодировок (например, если оборвана последовательность байт).
* Для этого можно воспользоваться модулем codecs.
* Также содержит классы для последовательного кодирования-декодирования и более подробной работы с типами кодировок.

In [29]:
import codecs

#### BOM

In [30]:
print(repr(codecs.BOM_UTF16_LE))
print(repr(codecs.BOM_UTF16_BE))
print(repr(codecs.BOM_UTF8)) # has no big sense for UTF-8

b'\xff\xfe'
b'\xfe\xff'
b'\xef\xbb\xbf'


Про удаление BOM:
* Декодирование из UTF-16 удаляет BOM автоматически.
* Но не из UTF-8: надо явно использовать s.decode('utf-8-sig')

### Свойства Unicode символов

In [31]:
import unicodedata

u = chr(233) + chr(0x0bf2) + chr(3972) + chr(6000) + chr(13231)

for i, c in enumerate(u):
    print(i, c, '%04x' % ord(c), unicodedata.category(c), end=" ")
    print(unicodedata.name(c))

# Get numeric value of second character
print(unicodedata.numeric(u[1]))

0 é 00e9 Ll LATIN SMALL LETTER E WITH ACUTE
1 ௲ 0bf2 No TAMIL NUMBER ONE THOUSAND
2 ྄ 0f84 Mn TIBETAN MARK HALANTA
3 ᝰ 1770 Lo TAGBANWA LETTER SA
4 ㎯ 33af So SQUARE RAD OVER S SQUARED
1000.0


* Категории: Letter, Number, Punctuation, Symbol
* Подкатегории зависят от категорий:
  * Ll (letter, lowercase)
  * No (number, other)
  * Mn (mark, nonspacing)
  * So (symbol, other)

### Общие советы по работе с кодировками

#### Декодируем, обрабатываем, кодируем

* Стараемся устраивать так, чтобы на вход приходил, по возможности, utf-8.
* Если по какой-то причине не так, то декодируем входные данные в str как можно раньше на входе.
* Используем внутри своей программы стандартный str (utf-8).
* Стремимся к тому, чтобы выходные данные в utf-8 принимались другими части системы.
* Если это не так, то кодируем из str только в самом конце перед передачей данных на выход программы.

#### Юникод и тесты

Если на вход или выход идут не utf-8 данные или внутри нужно использовать байты, то помещать проверки работы с байтами и строками в юнит тесты.

#### Угадывание и UTF-8 ftw

* По-умолчанию лучше предполагать (и использовать самому) utf-8.
* Дополнительно можно использовать byte order mark для угадывания кодировки.
* chardet.detect() от https://chardet.readthedocs.io/en/latest/usage.html

## utf-8 playground

In [32]:
def print_utf8_repr(unistr):
    utf8_bytes = unistr.encode('utf-8')
    for character in utf8_bytes:
        print(bin(character)[2:].zfill(8), end=" ")

In [33]:
print_utf8_repr(chr(127))

01111111 

In [34]:
print_utf8_repr(chr(128))

11000010 10000000 

In [35]:
print_utf8_repr(chr(4095))

11100000 10111111 10111111 

In [36]:
print_utf8_repr(chr(2048))

11100000 10100000 10000000 

In [37]:
print_utf8_repr(chr(2047))

11011111 10111111 

In [38]:
print_utf8_repr(chr(131071))

11110000 10011111 10111111 10111111 

In [39]:
print_utf8_repr(chr(65535))

11101111 10111111 10111111 

In [40]:
print_utf8_repr(chr(65536))

11110000 10010000 10000000 10000000 

0  
  
110 10  
1110 10  
11110 10  

#### Самостоятельная задача

Рекомендую попробовать написать функцию кодирования (декодирования) utf-8.

## Про проблемы кодировок в Python 2

Частично неактуально, но полезная часть про кодировки в общем и для понимания кода на python 2.7

* http://farmdev.com/talks/unicode/
* https://pythonhosted.org/kitchen/unicode-frustrations.html

* ASCII является кодировкой по-умолчанию в Python 2.
* Файлы могут содержать BOM (byte order mark).
* Не все внутренние части Python 2 поддерживают юникод.
* Невозможно точно угадать кодировку.
* Чтение чанками многобайтовых кодировок.

In [41]:
# Run with python 2 to see problems.
# This code won't work with python3 because 'unicode()' type was removed.

def process_line(line):
    line = unicode(line)
    return line.split(' ')[1]

line = "Hello, привет!\n"
print(process_line(line))

NameError: name 'unicode' is not defined

### -\*- coding: utf-8 -\*-

* Кодировка в Python 2 по-умолчанию - ASCII (в версии до 2.4 использовалась latin-1).
* Чтобы упростить запись спецсимволов в файле с исходным кодом, можно задать другую кодировку.
* Принято использовать соглашение, взятое из emacs.

In [ ]:
# -*- coding: utf-8 -*-

## Источники и детальнее про юникод и кодировки

* https://ru.wikipedia.org/wiki/Юникод
* https://docs.python.org/3.5/howto/unicode.html
* https://docs.python.org/3.5/library/codecs.html#standard-encodings